# Data Owner

In [ ]:
!pip install syft==0.5.0
!pip install tenseal

  Using cached syft-0.5.0-py2.py3-none-any.whl (504 kB)
  Using cached PyJWT-1.7.1-py2.py3-none-any.whl (18 kB)
  Using cached websocket_client-1.2.3-py3-none-any.whl (53 kB)
  Using cached dpcontracts-0.6.0.tar.gz (11 kB)
  Using cached names-0.3.0.tar.gz (789 kB)
  Using cached requests_toolbelt-0.9.1-py2.py3-none-any.whl (54 kB)
  Using cached aiortc-1.2.1-cp37-cp37m-manylinux2010_x86_64.whl (2.0 MB)
     |████████████████████████████████| 804.1 MB 2.5 kB/s 
  Using cached loguru-0.5.3-py3-none-any.whl (57 kB)
  Using cached torchcsprng-0.2.1-cp37-cp37m-manylinux1_x86_64.whl (4.9 MB)
  Using cached cryptography-36.0.1-cp36-abi3-manylinux_2_24_x86_64.whl (3.6 MB)
  Using cached forbiddenfruit-0.1.4.tar.gz (43 kB)
  Using cached syft_proto-0.5.3-py3-none-any.whl (66 kB)
  Using cached sqlitedict-1.7.0.tar.gz (28 kB)
  Using cached PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
  Using cached PyNaCl-1.5.0-cp36-abi3-m

In [ ]:
import syft as sy
import tenseal as ts
import torch
import pandas as pd
import random
import numpy as np

import pytest
from time import time
import matplotlib.pyplot as plt

In [ ]:
#Joining the Duet
duet = sy.duet("b46462b998a5a56ead9b311963fd6363")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 2e0cc9031e827b75f4e4bce36e4df3a1

♫♫♫ > ...waiting for partner to connect...


/usr/local/lib/python3.7/dist-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
/usr/local/lib/python3.7/dist-packages/aiortc/rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!


In [ ]:
#Defining The Model
class LR(torch.nn.Module):

    def __init__(self, n_features):
        super(LR, self).__init__()
        self.lr = torch.nn.Linear(n_features, 1)
        
    def forward(self, x):
        out = torch.sigmoid(self.lr(x))
        return out
    
class EncryptedLR:
    def __init__(self, torch_lr):
        # TenSEAL processes lists and not torch tensors
        # so we take out parameters from the PyTorch model
        self.weight = torch_lr.lr.weight.data.tolist()[0]
        self.bias = torch_lr.lr.bias.data.tolist()
        
    def forward(self, enc_x):
        # We don't need to perform sigmoid as this model
        # will only be used for evaluation, and the label
        # can be deduced without applying sigmoid
        enc_out = enc_x.dot(self.weight) + self.bias
        return enc_out
    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

In [ ]:
#Look at the data available in Duet
duet.store.pandas

""


In [ ]:
#Request the training data
#We request access to the training data.

x_train_ptr = duet.store["x_train"]
y_train_ptr = duet.store["y_train"]
x_train_ptr.request(reason="I would like to get the training data")
y_train_ptr.request(reason="I would like to get the training labels")


In [ ]:
duet.requests.pandas

""


In [ ]:
#Train the logistic regression

x_train = x_train_ptr.get(delete_obj=False)
y_train = y_train_ptr.get(delete_obj=False)

n_features = x_train.shape[1]
model = LR(n_features)

# use gradient descent with a learning_rate=1
optim = torch.optim.SGD(model.parameters(), lr=1)

# use Binary Cross Entropy Loss
criterion = torch.nn.BCELoss()

In [ ]:
EPOCHS = 30

def train(model, optim, criterion, x, y, epochs=EPOCHS):
    for e in range(1, epochs + 1):
        optim.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optim.step()
        print(f"Loss at epoch {e}: {loss.data}")
    return model

model = train(model, optim, criterion, x_train, y_train)

Loss at epoch 1: 0.7574891448020935
Loss at epoch 2: 0.6560847163200378
Loss at epoch 3: 0.6229860186576843
Loss at epoch 4: 0.6095389127731323
Loss at epoch 5: 0.6026040315628052
Loss at epoch 6: 0.5984874367713928
Loss at epoch 7: 0.5958308577537537
Loss at epoch 8: 0.5940183997154236
Loss at epoch 9: 0.5927305221557617
Loss at epoch 10: 0.5917855501174927
Loss at epoch 11: 0.5910735130310059
Loss at epoch 12: 0.5905246138572693
Loss at epoch 13: 0.5900930762290955
Loss at epoch 14: 0.5897480249404907
Loss at epoch 15: 0.5894678235054016
Loss at epoch 16: 0.5892370343208313
Loss at epoch 17: 0.5890448093414307
Loss at epoch 18: 0.5888829827308655
Loss at epoch 19: 0.5887452960014343
Loss at epoch 20: 0.5886272192001343
Loss at epoch 21: 0.5885251760482788
Loss at epoch 22: 0.588436484336853
Loss at epoch 23: 0.5883588194847107
Loss at epoch 24: 0.5882905125617981
Loss at epoch 25: 0.5882301330566406
Loss at epoch 26: 0.5881764888763428
Loss at epoch 27: 0.588128924369812
Loss at epoc